# UW Config Tool

The `uw config` tool has four modes:
- `realize` -- creates output that is as fully rendered as possible
- `compose` -- builds up a final config by repeatedly updating a base config with the contents of other configs of the same format
- `compare` -- compares two config files
- `validate` -- ensures that a given config file is structured properly

We'll focus on `realize` and `compare` CLI in the Intro, and will continue to see examples throughout the tutorial. API examples of these tools are available from the `uwtools` __[Config notebook](https://github.com/ufs-community/uwtools/blob/main/notebooks/config.ipynb)__.

# Realize

* Default format is YAML.
* Supported formats also include INI, Fortran Namelist, Bash.
* Can transform between compatible formats. All formats can be transformed to YAML.
  

In [43]:
%%bash
uw config realize -h  

usage: uw config realize [-h] [--version] [--input-file PATH]
                         [--input-format {ini,nml,sh,yaml}]
                         [--update-file PATH]
                         [--update-format {ini,nml,sh,yaml}]
                         [--output-file PATH]
                         [--output-format {ini,nml,sh,yaml}]
                         [--key-path KEY[.KEY...]] [--values-needed] [--total]
                         [--dry-run] [--quiet] [--verbose]

Realize config

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --input-file PATH, -i PATH
      Path to input file (defaults to stdin)
  --input-format {ini,nml,sh,yaml}
      Input format
  --update-file PATH, -u PATH
      Path to update file (defaults to stdin)
  --update-format {ini,nml,sh,yaml}
      Update format
  --output-file PATH, -o PATH
      Path to output file (defaults to stdout)
  --output-format {ini,nml,sh,yaml}
      Output format
  --key-path 

In [44]:
%%bash
uw config realize -i examples/simple.yaml

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '{{ cycle }}'


In [45]:
%%bash
echo 'cycle: !datetime 2025-09-24T12' | uw config realize -i examples/simple.yaml --update-format yaml

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '2025-09-24 12:00:00'
cycle: 2025-09-24T12:00:00


In [46]:
%%bash
echo 'cycle: !datetime 2025-09-24T12' \
  | uw config realize -i examples/simple.yaml --update-format yaml --key-path b

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


In [47]:
%%bash
echo 'cycle: !datetime 2025-09-24T12' \
  | uw config realize -i examples/simple.yaml --update-format yaml --key-path b -o b.yaml

In [48]:
%%bash
cat b.yaml

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


In [49]:
%%bash
args=(
    -i examples/simple.yaml 
    --update-format yaml
    --key-path b
    -o b.nml
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [50]:
%%bash
cat b.nml

&foo
    pi = 3.14
/

&bar
    one = 1
    two = 2
/


In [51]:
%%bash
args=(
    -i examples/simple.yaml 
    --update-format yaml
    --key-path b
    -o b.ini
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [52]:
%%bash
cat b.ini

[foo]
pi = 3.14

[bar]
one = 1
two = 2


# Compare

In [54]:
%%bash
uw config compare -h

usage: uw config compare --path1 PATH --path2 PATH [-h] [--version]
                         [--format1 {ini,nml,sh,yaml}]
                         [--format2 {ini,nml,sh,yaml}] [--quiet] [--verbose]

Compare configs

Required arguments:
  --path1 PATH
      Path to file 1
  --path2 PATH
      Path to file 2

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --format1 {ini,nml,sh,yaml}
      Format of file 1
  --format2 {ini,nml,sh,yaml}
      Format of file 2
  --quiet, -q
      Print no logging messages
  --verbose, -v
      Print all logging messages


In [55]:
%%bash
uw config compare --path1 examples/simple.nml --path2 b.nml

[2025-09-18T17:26:55]     INFO - examples/simple.nml
[2025-09-18T17:26:55]     INFO + b.nml
[2025-09-18T17:26:55]     INFO ---------------------------------------------------------------------
[2025-09-18T17:26:55]     INFO ↓ ? = info | -/+ = line unique to - or + file | blank = matching line
[2025-09-18T17:26:55]     INFO ---------------------------------------------------------------------
[2025-09-18T17:26:55]     INFO   bar:
[2025-09-18T17:26:55]     INFO     one: 1
[2025-09-18T17:26:55]     INFO -   three: 3
[2025-09-18T17:26:55]     INFO     two: 2
[2025-09-18T17:26:55]     INFO   foo:
[2025-09-18T17:26:55]     INFO -   pi: 3.1415
[2025-09-18T17:26:55]     INFO ?           --
[2025-09-18T17:26:55]     INFO +   pi: 3.14


CalledProcessError: Command 'b'uw config compare --path1 examples/simple.nml --path2 b.nml\n'' returned non-zero exit status 1.

In [56]:
%%bash
uw config compare --path1 b.nml --path2 b.nml

[2025-09-18T17:28:55]     INFO - b.nml
[2025-09-18T17:28:55]     INFO + b.nml
